## Code a Diffusion Model for a toy dataset

### Loading dependencies

In [ ]:
import sys
sys.path.append('../../..')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
import torch
from torch.utils.data import DataLoader
from probai.src.data.data import DataBatch
from probai.src.models.ddpm import DDPM
from probai.src.training.training_loop import Trainer


### Plot some samples for the Two Moons dataset

In [ ]:
def plot_samples(samples):
    plt.figure(figsize=(4, 4))
    plt.scatter(samples[:, 0], samples[:, 1], s=1, alpha=0.5)
    plt.xlim(-2, 2)
    plt.ylim(-2, 2)
    # remove the frame and ticks
    plt.box(False)
    plt.xticks([])
    plt.yticks([])


def get_toy_dataset(n):
    samples = datasets.make_moons(n_samples=n, noise=0.05)[0].astype(np.float32)
    samples -= np.array([[0.5, 0.25]])
    return samples


# plot the samples
plot_samples(get_toy_dataset(20000))

### Create dataloaders and initialize neural networks

In [ ]:
# create a data loader for the samples
batch_size = 256

def collate_to_batch(samples):
    x = torch.from_numpy(np.stack(samples, axis=0))
    batch = torch.LongTensor(range(x.shape[0]))
    return DataBatch(x=x, batch=batch)


train_loader = DataLoader(
    get_toy_dataset(50000), 
    batch_size=batch_size, 
    shuffle=True, 
    collate_fn=collate_to_batch, 
    drop_last=True
)
val_loader = DataLoader(
    get_toy_dataset(2000), 
    batch_size=batch_size, 
    shuffle=False, 
    collate_fn=collate_to_batch
)

# Define the neural network that defines the Score Function
class ResidualBlock(torch.nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.net = torch.nn.Sequential(
            torch.nn.Linear(dim, dim),
            torch.nn.ELU(),
            torch.nn.Linear(dim, dim)
        )
    def forward(self, x):
        return torch.nn.functional.elu(x + self.net(x))
    
class ScoreModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        dimh = 256
        self.epsilon_predictor = torch.nn.Sequential(
            torch.nn.Linear(3, dimh),
            torch.nn.ELU(),
            ResidualBlock(dimh),
            ResidualBlock(dimh),
            torch.nn.Linear(dimh, 2)
        )
        
    def forward(self, z_t, t, **kwargs):
        # concatenate the input with the continuous time
        z_t = torch.cat([z_t, t.unsqueeze(1) * 2 - 1], dim=1)
        
        # predict the score
        score = self.epsilon_predictor(z_t)
        return score
    


### <span style="color:orange;">TO DO:</span> Code the missing parts in DDPM


In [1]:
### Include here the instrucitons of what they need to code

### Train the DDPM model and plot generated samples

In [ ]:

N = 1000
ddpm = DDPM(noise_schedule_type="linear", model=ScoreModel(), N=N)
trainer = Trainer(ddpm, lr=0.001)
trainer.train(train_loader, val_loader, epochs=100, device=torch.device('cuda:0'))


In [ ]:
# Plot val loss
plt.plot(trainer.val_losses)

In [ ]:
# Plot generated samples
samples = ddpm.sample([10000, 2])
plot_samples(samples)